<h3> BERT classifier v6 trained on balanced data for male/female in toxic class, label = gender, tested gender predictions for toxic class, visualise words with higher gradients relied on for gender classification, adjusted how test dataset created</h3>
<p>Using code from: https://mccormickml.com/2019/07/22/BERT-fine-tuning/, 
https://captum.ai/tutorials/IMDB_TorchText_Interpret</p>

In [1]:
# !pip install captum

In [2]:
CUDA_LAUNCH_BLOCKING="1"

In [3]:
# import libraries
import pandas as pd
import numpy as np
#!conda install -y tensorflow
#!conda install -y pytorch torchvision -c pytorch
#!pip install transformers
# import tensorflow as tf
import torch
import math

import spacy

import torchtext
import torchtext.data
import torch.nn as nn
import torch.nn.functional as F

from torchtext.vocab import Vocab

from captum.attr import LayerIntegratedGradients, TokenReferenceBase, visualization

nlp = spacy.load('en_core_web_sm')

# from google.colab import drive
# drive.mount('/content/drive')

In [4]:
# read files and create dataframe
toxicity_comments = pd.read_csv('toxicity_annotated_comments.tsv', sep = '\t', index_col = 0)
toxicity_annotations = pd.read_csv('toxicity_annotations.tsv',  sep = '\t')
toxicity_demographics = pd.read_csv('toxicity_worker_demographics.tsv', sep = '\t')

toxicity = toxicity_comments.merge(toxicity_annotations, how ='outer', on="rev_id")
toxicity = toxicity.merge(toxicity_demographics, how ='outer', on="worker_id").sort_values(by=['rev_id','worker_id'])

# remove newline and tab tokens
toxicity['comment'] = toxicity['comment'].apply(lambda x: x.replace("NEWLINE_TOKEN", " "))
toxicity['comment'] = toxicity['comment'].apply(lambda x: x.replace("TAB_TOKEN", " "))

# add binary gender columns
toxicity = toxicity[toxicity['gender']!='other']
toxicity = pd.concat([toxicity, pd.get_dummies(toxicity.gender).rename(columns = "{}_binary".format)], axis = 1)

# limit size of dataset for testing purposes
very_toxic = toxicity[toxicity.toxicity_score == -2.0]
toxic = toxicity[toxicity.toxicity_score == -1.0]
neutral = toxicity[toxicity.toxicity_score == 0.0]
nontoxic = toxicity[toxicity.toxicity_score == 1.0]
healthy = toxicity[toxicity.toxicity_score == 2.0]

female_toxic = toxic[toxic.female_binary == 1]
male_toxic = toxic[toxic.male_binary == 1]
female_very_toxic = very_toxic[very_toxic.female_binary == 1]
male_very_toxic = very_toxic[very_toxic.male_binary == 1]

size = min(female_toxic.shape[0],male_toxic.shape[0],female_very_toxic.shape[0],male_very_toxic.shape[0])
#size = 2000
print(size)

#split into train and test sets
test_size = math.floor(size*0.2)
train_size = math.ceil(size*0.8)

#create test dataset and drop from dataframes so not also chosen for training
female_toxic_test = female_toxic.sample(n=test_size)
female_toxic = female_toxic.drop(female_toxic_test.index)
male_toxic_test = male_toxic.sample(n=test_size)
male_toxic = male_toxic.drop(male_toxic_test.index)
female_vtoxic_test = female_very_toxic.sample(n=test_size)
female_very_toxic = female_very_toxic.drop(female_vtoxic_test.index)
male_vtoxic_test = male_very_toxic.sample(n=test_size)
male_very_toxic = male_very_toxic.drop(male_vtoxic_test.index)
test_data = pd.concat([female_toxic_test, male_toxic_test,female_vtoxic_test, male_vtoxic_test])
test_data = test_data.sample(frac=1)
print(test_data.shape[0])


toxic_data = pd.concat([female_toxic.sample(train_size), male_toxic.sample(train_size),female_very_toxic.sample(train_size), male_very_toxic.sample(train_size)])
print(toxic_data.shape[0])

display (toxic_data.head(5))

14076
11260
45044


,rev_id,comment,year,logged_in,ns,sample,split,worker_id,toxicity,toxicity_score,gender,english_first_language,age_group,education,female_binary,male_binary
1324313,74200889.0,==Ban this vandal== http://en.wikipedia.org/...,2006,False,article,blocked,train,221,1,-1.0,female,0.0,30-45,masters,1,0
1535489,627226989.0,Grandma Terri should burn alive and be shot!...,2014,False,user,blocked,dev,1102,1,-1.0,female,0.0,18-30,bachelors,1,0
937299,624363827.0,== polish Nobility == your list is very sho...,2014,False,user,random,train,1328,1,-1.0,female,0.0,18-30,masters,1,0
864508,465488942.0,` ::::::::::::Hopefully you don't have the sho...,2011,True,user,blocked,test,990,1,-1.0,female,0.0,45-60,professional,1,0
1007818,198301437.0,== PEOPLE ON THIS SITE SUCK == FUCKING BAN...,2008,False,user,blocked,test,1261,1,-1.0,female,0.0,30-45,masters,1,0


In [5]:
# try to use gpu
if torch.cuda.is_available():      
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: GeForce RTX 2070


In [6]:
# extract relevant information
comments = toxic_data.comment.values
labels = toxic_data.female_binary.values

In [7]:
! pip install transformers==3.5.1
from transformers import BertTokenizer

# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [8]:
# Tokenize all of the sentences and map the tokens to their word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for cmt in comments:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        cmt,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 250,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Print sentence 0, now as a list of IDs.
# print('Original: ', comments[0],scores[0])
# print('Token IDs:', input_ids[0])

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/home/nick/anaconda3/envs/nlp/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [9]:
from torch.utils.data import TensorDataset, random_split

# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))

40,539 training samples
4,505 validation samples


In [10]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

# The DataLoader needs to know our batch size for training, so we specify it 
# here. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.
batch_size = 8

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size
)

In [11]:
from transformers import BertForSequenceClassification, AdamW, BertConfig

# Load BertForSequenceClassification, the pretrained BERT model with a single 
# linear classification layer on top. 
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased", # Use the 12-layer BERT model, with an uncased vocab.
    num_labels = 2, # The number of output labels--2 for binary classification.
                    # You can increase this for multi-class tasks.   
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = False, # Whether the model returns all hidden-states.
)

# Tell pytorch to run this model on the GPU.
model = model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [12]:
#defining fields
TEXT = torchtext.data.Field(lower=True, tokenize='spacy', tokenizer_language='en_core_web_sm')
Label = torchtext.data.LabelField(dtype = torch.float)
fields = [('text',TEXT),('label',Label)]

/home/nick/anaconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: Field class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)
/home/nick/anaconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/field.py:150: UserWarning: LabelField class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('{} class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.'.format(self.__class__.__name__), UserWarning)


In [13]:
# pre-processing comments for vocab
new_comments = []
for comment in comments:
    text = [tok.text for tok in nlp.tokenizer(comment)]
    text = [word.strip(' ') for word in text]
    text = [word.lower() for word in text]
    while '' in text:
      text.remove('')
    new_comments.append(' '.join(text))

In [14]:
examples = [[comments[i],labels[i]] for i in range(len(comments))]
formatted_examples = []
for example in examples:
    new_example = torchtext.data.Example.fromlist(example,fields)
    formatted_examples.append(new_example)
train = torchtext.data.Dataset(formatted_examples, fields, filter_pred=None)

/home/nick/anaconda3/envs/nlp/lib/python3.8/site-packages/torchtext/data/example.py:78: UserWarning: Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.
  warnings.warn('Example class will be retired soon and moved to torchtext.legacy. Please see the most recent release notes for further information.', UserWarning)


In [15]:
#setting up vocab for word embeddings
from torchtext import vocab

loaded_vectors = vocab.GloVe(name='6B', dim=50)

In [16]:
TEXT.build_vocab(train, vectors=loaded_vectors, max_size=len(loaded_vectors.stoi))
    
TEXT.vocab.set_vectors(stoi=loaded_vectors.stoi, vectors=loaded_vectors.vectors, dim=loaded_vectors.dim)
Label.build_vocab(['man','woman'])
#Label.build_vocab(train)
print('Vocabulary Size: ', len(TEXT.vocab))

Vocabulary Size:  62785


In [17]:
#provide padding index as reference token index
PAD_IND = TEXT.vocab.stoi['pad']
token_reference = TokenReferenceBase(reference_token_idx=PAD_IND)

In [18]:
# Note: AdamW is a class from the huggingface library (as opposed to pytorch) 
# I believe the 'W' stands for 'Weight Decay fix"
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )

In [19]:
from transformers import get_linear_schedule_with_warmup

# Number of training epochs. The BERT authors recommend between 2 and 4. 
# We chose to run for 4, but we'll see later that this may be over-fitting the
# training data.
epochs = 3

# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0, # Default value in run_glue.py
                                            num_training_steps = total_steps)

In [20]:
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [21]:
import time
import datetime

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [22]:
import random

pretrained = True

if pretrained:
    model.load_state_dict(torch.load('bert_model.pt'))
    model.eval()
else:

  # This training code is based on the `run_glue.py` script here:
  # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

  # Set the seed value all over the place to make this reproducible.
  seed_val = 42

  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)

  # We'll store a number of quantities such as training and validation loss, 
  # validation accuracy, and timings.
  training_stats = []

  # Measure the total training time for the whole run.
  total_t0 = time.time()

  # For each epoch...
  for epoch_i in range(0, epochs):
      
      # ========================================
      #               Training
      # ========================================
      
      # Perform one full pass over the training set.

      print("")
      print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
      print('Training...')

      # Measure how long the training epoch takes.
      t0 = time.time()

      # Reset the total loss for this epoch.
      total_train_loss = 0

      # Put the model into training mode. Don't be misled--the call to 
      # `train` just changes the *mode*, it doesn't *perform* the training.
      # `dropout` and `batchnorm` layers behave differently during training
      # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
      model.train()

      # For each batch of training data...
      for step, batch in enumerate(train_dataloader):

          # Progress update every 40 batches.
          if step % 40 == 0 and not step == 0:
              # Calculate elapsed time in minutes.
              elapsed = format_time(time.time() - t0)
              
              # Report progress.
              print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using the 
          # `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)

          # Always clear any previously calculated gradients before performing a
          # backward pass. PyTorch doesn't do this automatically because 
          # accumulating the gradients is "convenient while training RNNs". 
          # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
          model.zero_grad()        

          # Perform a forward pass (evaluate the model on this training batch).
          # The documentation for this `model` function is here: 
          # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
          # It returns different numbers of parameters depending on what arguments
          # arge given and what flags are set. For our useage here, it returns
          # the loss (because we provided labels) and the "logits"--the model
          # outputs prior to activation.
          loss, logits = model(b_input_ids, 
                              token_type_ids=None, 
                              attention_mask=b_input_mask, 
                              labels=b_labels.long())

          # Accumulate the training loss over all of the batches so that we can
          # calculate the average loss at the end. `loss` is a Tensor containing a
          # single value; the `.item()` function just returns the Python value 
          # from the tensor.
          total_train_loss += loss.item()

          # Perform a backward pass to calculate the gradients.
          loss.backward()

          # Clip the norm of the gradients to 1.0.
          # This is to help prevent the "exploding gradients" problem.
          torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

          # Update parameters and take a step using the computed gradient.
          # The optimizer dictates the "update rule"--how the parameters are
          # modified based on their gradients, the learning rate, etc.
          optimizer.step()

          # Update the learning rate.
          scheduler.step()

      # Calculate the average loss over all of the batches.
      avg_train_loss = total_train_loss / len(train_dataloader)            
      
      # Measure how long this epoch took.
      training_time = format_time(time.time() - t0)

      print("")
      print("  Average training loss: {0:.2f}".format(avg_train_loss))
      print("  Training epoch took: {:}".format(training_time))
          
      # ========================================
      #               Validation
      # ========================================
      # After the completion of each training epoch, measure our performance on
      # our validation set.

      print("")
      print("Running Validation...")

      t0 = time.time()

      # Put the model in evaluation mode--the dropout layers behave differently
      # during evaluation.
      model.eval()

      # Tracking variables 
      total_eval_accuracy = 0
      total_eval_loss = 0
      nb_eval_steps = 0

      # Evaluate data for one epoch
      for batch in validation_dataloader:
          
          # Unpack this training batch from our dataloader. 
          #
          # As we unpack the batch, we'll also copy each tensor to the GPU using 
          # the `to` method.
          #
          # `batch` contains three pytorch tensors:
          #   [0]: input ids 
          #   [1]: attention masks
          #   [2]: labels 
          b_input_ids = batch[0].to(device)
          b_input_mask = batch[1].to(device)
          b_labels = batch[2].to(device)
          
          # Tell pytorch not to bother with constructing the compute graph during
          # the forward pass, since this is only needed for backprop (training).
          with torch.no_grad():        

              # Forward pass, calculate logit predictions.
              # token_type_ids is the same as the "segment ids", which 
              # differentiates sentence 1 and 2 in 2-sentence tasks.
              # The documentation for this `model` function is here: 
              # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
              # Get the "logits" output by the model. The "logits" are the output
              # values prior to applying an activation function like the softmax.
              (loss, logits) = model(b_input_ids, 
                                    token_type_ids=None, 
                                    attention_mask=b_input_mask,
                                    labels=b_labels.long())
              
          # Accumulate the validation loss.
          total_eval_loss += loss.item()

          # Move logits and labels to CPU
          logits = logits.detach().cpu().numpy()
          label_ids = b_labels.to('cpu').numpy()

          # Calculate the accuracy for this batch of test sentences, and
          # accumulate it over all batches.
          total_eval_accuracy += flat_accuracy(logits, label_ids)
          

      # Report the final accuracy for this validation run.
      avg_val_accuracy = total_eval_accuracy / len(validation_dataloader)
      print("  Accuracy: {0:.2f}".format(avg_val_accuracy))

      # Calculate the average loss over all of the batches.
      avg_val_loss = total_eval_loss / len(validation_dataloader)
      
      # Measure how long the validation run took.
      validation_time = format_time(time.time() - t0)
      
      print("  Validation Loss: {0:.2f}".format(avg_val_loss))
      print("  Validation took: {:}".format(validation_time))

      # Record all statistics from this epoch.
      training_stats.append(
          {
              'epoch': epoch_i + 1,
              'Training Loss': avg_train_loss,
              'Valid. Loss': avg_val_loss,
              'Valid. Accur.': avg_val_accuracy,
              'Training Time': training_time,
              'Validation Time': validation_time
          }
      )

  print("")
  print("Training complete!")

  print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))
  torch.save(model.state_dict(), 'bert_model.pt')

  # Display floats with two decimal places.
  pd.set_option('precision', 2)

  # Create a DataFrame from our training statistics.
  df_stats = pd.DataFrame(data=training_stats)

  # Use the 'epoch' as the row index.
  df_stats = df_stats.set_index('epoch')

  # A hack to force the column headers to wrap.
  #df = df.style.set_table_styles([dict(selector="th",props=[('max-width', '70px')])])

  # Display the table.
  df_stats

In [25]:
# #examining test data

# correct_female_pred = []
# correct_male_pred = []
# accuracies = []

# df = test_data

# df = df.sample(frac=1)
# print('Number of test sentences: {:,}\n'.format(df.shape[0]))

# comments = df.comment.values
# labels = df.female_binary.values

# # Tokenize all of the sentences and map the tokens to thier word IDs.
# input_ids = []
# attention_masks = []

# # For every sentence...
# for cmt in comments:
#     # `encode_plus` will:
#     #   (1) Tokenize the sentence.
#     #   (2) Prepend the `[CLS]` token to the start.
#     #   (3) Append the `[SEP]` token to the end.
#     #   (4) Map tokens to their IDs.
#     #   (5) Pad or truncate the sentence to `max_length`
#     #   (6) Create attention masks for [PAD] tokens.
#     encoded_dict = tokenizer.encode_plus(
#                         cmt,                      # Sentence to encode.
#                         add_special_tokens = True, # Add '[CLS]' and '[SEP]'
#                         max_length = 250,           # Pad & truncate all sentences.
#                         pad_to_max_length = True,
#                         return_attention_mask = True,   # Construct attn. masks.
#                         return_tensors = 'pt',     # Return pytorch tensors.
#                   )
      
#     # Add the encoded sentence to the list.    
#     input_ids.append(encoded_dict['input_ids'])
      
#     # And its attention mask (simply differentiates padding from non-padding).
#     attention_masks.append(encoded_dict['attention_mask'])

# # Convert the lists into tensors.
# input_ids = torch.cat(input_ids, dim=0)
# attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(labels)

# # Set the batch size.  
# batch_size = 8

# # Create the DataLoader.
# prediction_data = TensorDataset(input_ids, attention_masks, labels)
# prediction_sampler = SequentialSampler(prediction_data)
# prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

# # Prediction on test set

# print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# # Put model in evaluation mode
# model.eval()

# # Tracking variables 
# predictions , true_labels = [], []

# # Predict 
# for batch in prediction_dataloader:
#   # Add batch to GPU
#   batch = tuple(t.to(device) for t in batch)
    
# # Unpack the inputs from our dataloader
#   b_input_ids, b_input_mask, b_labels = batch
    
#   # Telling the model not to compute or store gradients, saving memory and 
#   # speeding up prediction
#   with torch.no_grad():
#       # Forward pass, calculate logit predictions
#       outputs = model(b_input_ids, token_type_ids=None, 
#                         attention_mask=b_input_mask)

#   logits = outputs[0]

#   # Move logits and labels to CPU
#   logits = logits.detach().cpu().numpy()
#   label_ids = b_labels.to('cpu').numpy()
    
#   # Store predictions and true labels
#   predictions.append(logits)
#   true_labels.append(label_ids)

# print('    DONE.')

# from sklearn.metrics import f1_score, confusion_matrix, accuracy_score
# import seaborn as sns
# import matplotlib.pyplot as plt
  
# # Combine the results across all batches. 
# flat_pred = np.concatenate(predictions, axis=0)
# # For each sample, pick the label (0 or 1) with the higher score.
# flat_predictions = np.argmax(flat_pred, axis=1).flatten()

# # Combine the correct labels for each batch into a single list.
# flat_true_labels = np.concatenate(true_labels, axis=0)

#   for i in range(len(flat_true_labels)):
#     if flat_true_labels[i]==flat_predictions[i]:
#       if flat_true_labels[i] == 1:
#         correct_female_pred.append(tuple(flat_pred[i]))
#       else:
#         correct_male_pred.append(tuple(flat_pred[i]))

# f1 = f1_score(flat_true_labels, flat_predictions)
# print('Total F1: %.3f' % f1)
# acc = accuracy_score(flat_true_labels,flat_predictions)
# print('Accuracy: %.2f' % acc)
# accuracies.append(acc)

# cf_matrix = confusion_matrix(flat_true_labels, flat_predictions)
# group_names = ['True Neg','False Pos','False Neg','True Pos']
# group_counts = ['{0:0.0f}'.format(value) for value in
#                 cf_matrix.flatten()]
# group_percentages = ['{0:.2%}'.format(value) for value in
#                   cf_matrix.flatten()/np.sum(cf_matrix)]
# labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
#           zip(group_names,group_counts,group_percentages)]
# try:
#   labels = np.asarray(labels).reshape(2,2)
#   sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Blues')
#   plt.show()
# except ValueError:
#   print("could not display")


In [24]:
# import plotly.graph_objects as go

# unzipped_object = zip(*correct_female_pred)
# unzipped_list = list(unzipped_object)

# fig = go.Figure(data=go.Scatter(x=unzipped_list[1],
#                                 y=unzipped_list[0],
#                                 mode='markers'))

# fig.update_layout(title='Likelihood of genders for true positive (female)',
#                   xaxis_title="Likelihood female",
#                   yaxis_title="Likelihood male")
# fig.show()

# unzipped_object = zip(*correct_male_pred)
# unzipped_list = list(unzipped_object)

# fig = go.Figure(data=go.Scatter(x=unzipped_list[0],
#                                 y=unzipped_list[1],
#                                 mode='markers'))

# fig.update_layout(title='Likelihood of genders for true negative (male)',
#                   xaxis_title="Likelihood male",
#                   yaxis_title="Likelihood female")
# fig.show()

In [26]:
def squad_pos_forward_func(inputs, token_type_ids=None, position_ids=None, attention_mask=None, position=0):
    pred = predict(inputs,
                   token_type_ids=token_type_ids,
                   position_ids=position_ids,
                   attention_mask=attention_mask)
    pred = pred[position]
    return pred.max(1).values

In [27]:
lig = LayerIntegratedGradients(squad_pos_forward_func, model.bert.embeddings)

In [28]:
#forward function that supports sigmoid
def forward_with_sigmoid(input):
    return torch.sigmoid(model(input))

In [29]:
#A helper function to perform forward pass of the model and make predictions
def predict(inputs, token_type_ids=None, position_ids=None, attention_mask=None):
    return model(inputs, token_type_ids=token_type_ids,
                position_ids=position_ids, attention_mask=attention_mask, )

In [30]:
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/nick/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
# interpret_sentence(model, 'It was a fantastic performance !', label=1)

import string

df = test_data

df = df.sample(frac=1)

df['comment'] = df['comment'].str.translate(str.maketrans('', '', string.punctuation))
df['comment'] = df['comment'].replace('\n',' ', regex=True)

comments = df.comment.values
labels = df.female_binary.values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for cmt in comments:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        cmt,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 250,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                  )
      
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
      
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
tensor_labels = torch.tensor(labels)

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, tensor_labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)

# Put model in evaluation mode
model.eval()
    
# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
  batch = tuple(t.to(device) for t in batch)
    
# Unpack the inputs from our dataloader
  b_input_ids, b_input_mask, b_labels = batch
    
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
  with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, token_type_ids=None, 
                        attention_mask=b_input_mask)

  logits = outputs[0]

  # Move logits and labels to CPU
  logits = logits.detach().cpu().numpy()
  label_ids = b_labels.to('cpu').numpy()
    
  # Store predictions and true labels
  predictions.append(logits)
  true_labels.append(label_ids)

# Combine the results across all batches. 
flat_pred = np.concatenate(predictions, axis=0)
# For each sample, pick the label (0 or 1) with the higher score.
flat_predictions = np.argmax(flat_pred, axis=1).flatten()
# Combine the correct labels for each batch into a single list.
flat_true_labels = np.concatenate(true_labels, axis=0)    

/home/nick/anaconda3/envs/nlp/lib/python3.8/site-packages/transformers/tokenization_utils_base.py:2149: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [32]:
# accumulate couple samples in this array for visualization purposes
vis_data_records_ig = []

def interpret_sentence(model, sentence, pred, flat_pred, min_len = 18, label = 0):
    text = [tok.text for tok in nlp.tokenizer(sentence)]
    text = [word.strip(' ') for word in text]
    while '' in text:
      text.remove('')
    text = [word.lower() for word in text]
    #text = [word.lower() for word in text if not word.lower() in stopwords.words('english')]
    if len(text) < min_len:
        text += ['pad'] * (min_len - len(text))
    else:
      text = text[:min_len]
    print(text)
    
    indexed = [TEXT.vocab.stoi[t] for t in text]
    print(indexed)

    model.zero_grad()
    
    # preventing CUDA error
    new_indices = []
    for index in indexed:
        if index>30000:
            index = 0
        new_indices.append(index)
    indexed = new_indices

    input_indices = torch.tensor(indexed, device=device)
    input_indices = input_indices.unsqueeze(0)
    
    # input_indices dim: [sequence_length]
    seq_length = min_len
    #seq_length = len(text)

    # predict
    #pred = forward_with_sigmoid(input_indices).item()
#     pred = predict(input_indices)
    print(flat_pred)
    print(pred)

    pred_ind = pred
    prob = np.amax(flat_pred)

    # generate reference indices for each sample
    reference_indices = token_reference.generate_reference(seq_length, device=device).unsqueeze(0)
    
#     token_reference2 = TokenReferenceBase(reference_token_idx=PAD_IND+10000)
#     input_indices = token_reference2.generate_reference(seq_length, device=device).unsqueeze(0)

    print(input_indices)
    print(reference_indices)

    # compute attributions and approximation delta using layer integrated gradients
    attributions_ig, delta = lig.attribute(input_indices, reference_indices, \
                                           n_steps=500, return_convergence_delta=True)

    print('pred: ', Label.vocab.itos[pred_ind], '(', '%.2f'%prob, ')', ', delta: ', abs(delta),'\n')
    add_attributions_to_visualizer(attributions_ig, text, prob, pred_ind, label, delta, vis_data_records_ig)
    
def add_attributions_to_visualizer(attributions, text, pred, pred_ind, label, delta, vis_data_records):
    attributions = attributions.sum(dim=2).squeeze(0)
    attributions = attributions / torch.norm(attributions)
    attributions = attributions.cpu().detach().numpy()

    # storing couple samples in an array for visualization purposes
    vis_data_records.append(visualization.VisualizationDataRecord(
                            attributions,
                            pred,
                            Label.vocab.itos[pred_ind],
                            Label.vocab.itos[label],
                            Label.vocab.itos[1],
                            attributions.sum(),       
                            text,
                            delta))

In [33]:
import random

vis_data_records_ig = []

for i in range(10):
    num = random.randrange(len(comments))
    print("\n",labels[num])
    interpret_sentence(model,comments[num],flat_predictions[num],flat_pred[num],label=flat_true_labels[num])


 1
['tu', 'mom', 'es', 'a', 'dirty', 'slut', 'disgusting', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad']
[11093, 838, 11441, 11, 445, 1277, 1180, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484]
[ 0.07579865 -0.18785585]
0
tensor([[11093,   838, 11441,    11,   445,  1277,  1180, 26484, 26484, 26484,
         26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484]],
       device='cuda:0')
tensor([[26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484,
         26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484]],
       device='cuda:0')
pred:  man ( 0.08 ) , delta:  tensor([9.8341e-08], device='cuda:0', dtype=torch.float64) 


 0
['dude', 'calm', 'the', 'fuck', 'down', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad', 'pad']
[683, 2185, 6, 20, 171, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484, 26484]
[ 0.10271511 -0.24984382]
0
tensor([

In [34]:
print('Visualize attributions based on Integrated Gradients')
html=visualization.visualize_text(vis_data_records_ig)

Visualize attributions based on Integrated Gradients


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
woman,man (0.08),woman,1.91,tu mom es a dirty slut disgusting pad pad pad pad pad pad pad pad pad pad pad
man,man (0.10),woman,0.22,dude calm the fuck down pad pad pad pad pad pad pad pad pad pad pad pad pad
man,man (0.02),woman,1.35,guess what your retarded pad pad pad pad pad pad pad pad pad pad pad pad pad pad
woman,woman (0.79),woman,1.75,dear josiah 2 random blaggards named bastin and john carter have repeatedly vandalized my page and i solemnly
woman,woman (0.48),woman,-0.07,yes any third party would be able to tell you your grounds for removing that content is not
woman,man (0.09),woman,1.27,hi jim1138 before you move your mouse over the delete button have a think about this you arrogant
woman,woman (0.79),woman,0.16,i have to disagree with you on several key issues for example the horses mouth is extremely likely
woman,man (0.03),woman,0.30,lol you and your king lol can be neutral as fuck but that wo nt save you if
woman,man (0.13),woman,-0.34,dear mr anal retentive wtf are you talking about bitch that was a perfect edit the topic was
man,man (0.03),woman,-0.82,247 stop making idiotic changes to the article its bad enough having that fool hholt1 doing it pad
